TODO:
- Code up helper functions for steering
- Code up helper functions for neuron reactivation
- Code up visualization functs

Down the line TODO:
- Other AE architectures -- FSQAE, VAE/NVAE (?)
- Other GPT models -- LLaMA, Mistral (need to tokenize fineweb with sentencepiece)
- Does less context length give similar feature fidelity? 

In [ ]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1"

import torch
import torch.nn as nn
import torch.nn.functional as F


loading weights from pretrained gpt: gpt2-medium
GPT2 model finished loading


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (h): ModuleList(
      (0-23): 24 x Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1024, out_features=3072, bias=True)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [ ]:
#Load in base model

"""
GPT medium -- SAE is 32768
GPT (small) -- SAE is 8192 (for simplicty)
"""

from gpt2_model import * 

model_name = 'gpt2'
device = "mps:0"

model = GPT.from_pretrained('gpt2-medium')
model.to(device)

#Freeze params, eval mode
for name, param in model.named_parameters():
    param.requires_grad = False
model.eval()


In [ ]:
#Load in SAE
from SAE_model import *

model_name = 'gpt2-small'
run_name = 'trial'
load_weights = True

config = GPT_Config
h_dim = 8192
sae_layer = config.n_layer // 2

sae_model = SAE(GPT_medium_Config, hidden_dim=h_dim)
sae_model.to(device)

# Load in pretrained weights
weights_dir = os.path.join(os.path.dirname(__file__), 'weights')
weight_file = os.path.join(weights_dir, f"{model_name}_{run_name}_layer{sae_layer}.ckpt")
if os.path.exists(weight_file):
    sae_model.load_state_dict(torch.load(weight_file, map_location=device, weights_only=True))


/var/folders/13/8p96hpz16tx9325nkkzrn29c0000gn/T/ipykernel_28145/815066745.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sae_model.load_state_dict(torch.load(weight_f

In [ ]:
# Same val loader as used during training
from data_loader import *

data_dir = os.path.join(os.path.dirname(__file__), 'fineweb-edu-10b')
data_files = os.listdir(data_dir)
shards = [os.path.join(data_dir,shard) for shard in data_files]

val_shards = shards[:5]
B = 8 #Batch size
T = 1024 #Sequence length

val_loader = DataLoader(B, T, val_shards)


In [3]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')

def get_top_sae_activations(text):
    return NotImplementedError


In [ ]:
def generate_text():
    return NotImplementedError


In [ ]:
def generate_text_with_steering():
    return NotImplementedError
